In [14]:
import ipywidgets as widgets
from IPython.display import display 
from IPython.display import Image
import ipympl
import matplotlib
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np

In [15]:
from dashboard_auxiliary_functions import process_df_for_widget, hover, get_traffic_matrix, \
                                          get_egress_traffic, get_ingress_traffic, \
                                          apply_changes, summarize_change_html, load_balancing, \
                                          simple_load_balancing, get_total_per_link, extract_values_from_json_object, \
                                          process_data 

# Data loading

In [16]:
# load traffic matrix
traffic_matrix = pd.read_pickle("preparing_data/traffic_matrix.pickle")

In [17]:
# calculate the "routing table". 
# In our simple example, we first calculate every src net to the scr asn
# and assume that every link to that srcasn can be used to reach the src net.
# In a real deployement, the routing table is calcualted using the BGP table.
routing_table = {}
links_per_as = {}
links_to_as = {}

for dst_as, link in traffic_matrix[["DST_AS", "EGRESS_LINK"]].drop_duplicates().values:
    links_to_as[link] = dst_as
    links_per_as.setdefault(dst_as, set()).add(link)
    
for src_net, src_as in traffic_matrix[["SRC_NET", "SRC_AS"]].drop_duplicates().values:
    routing_table.setdefault(src_net, set()).update(links_per_as[src_as])


In [18]:
# The next jsons are "fake" configurations for all routers.
# I took the model from https://github.com/CiscoDevNet/openconfig-getting-started/blob/master/models/bgp/
# In the near future, we hope we can get configs like this by routers supporting open config yang models
bgp_json = '''
{
 "bgp:bgp": {
  "global": {
   "state": {
    "as": 65001,
    "total-paths": 2,
    "total-prefixes": 2
   },
   "afi-safis": {
    "afi-safi": [
     {
      "afi-safi-name": "ipv4-unicast",
      "state": {
       "afi-safi-name": "ipv4-unicast",
       "enabled": true,
       "total-paths": 2,
       "total-prefixes": 2
      }
     }
    ]
   }
  },
  "peer-groups": {
   "peer-group": [
    {
     "peer-group-name": "IBGP",
     "state": {
      "peer-group-name": "IBGP",
      "peer-as": 65001
     },
     "transport": {
      "state": {
       "local-address": "Loopback0"
      }
     },
     "afi-safis": {
      "afi-safi": [
       {
        "afi-safi-name": "ipv4-unicast",
        "state": {
         "afi-safi-name": "ipv4-unicast",
         "enabled": true
        },
        "apply-policy": {
         "state": {
          "export-policy": [
           "POLICY2"
          ]
         }
        }
       }
      ]
     }
    }
   ]
  },
  "neighbors": {
   "neighbor": [
    {
     "neighbor-address": "172.16.255.3",
     "state": {
      "neighbor-address": "172.16.255.3",
      "peer-group": "IBGP",
      "queues": {
       "input": 0,
       "output": 0
      },
      "session-state": "bgp-st-estab",
      "supported-capabilities": [
       "MPBGP"
      ],
      "messages": {
       "sent": {
        "NOTIFICATION": 0,
        "UPDATE": 1
       },
       "received": {
        "NOTIFICATION": 0,
        "UPDATE": 3
       }
      }
     },
     "transport": {
      "state": {
       "local-port": 21344,
       "remote-address": "172.16.255.3",
       "remote-port": 179
      }
     },
     "timers": {
      "state": {
       "negotiated-hold-time": 180
      }
     },
     "afi-safis": {
      "afi-safi": [
       {
        "afi-safi-name": "ipv4-unicast",
        "state": {
         "active": true,
         "prefixes": {
          "received": 2,
          "sent": 0
         }
        }
       }
      ]
     },
     "graceful-restart": {
      "state": {
       "peer-restart-time": 120
      }
     }
    }
   ]
  }
 }
}
'''

policy_code = '''{
 "routing-policy:routing-policy": {
  "defined-sets": {
   "bgp-policy:bgp-defined-sets": {
    "as-path-sets": {
     "as-path-set": [
      {
       "as-path-set-name": "AS-PATH-SET1",
       "as-path-set-member": [
        "^65172"
       ]
      }
     ]
    },
    "community-sets": {
     "community-set": [
      {
       "community-set-name": "COMMUNITY-SET1",
       "community-member": [
        "ios-regex '^65172:17...$'",
        "65172:16001"
       ]
      }
     ]
    }
   }
  },
  "policy-definitions": {
   "policy-definition": [
    {
     "name": "POLICY2",
     "statements": {
      "statement": [
       {
        "name": "community-set1",
        "conditions": {
         "bgp-policy:bgp-conditions": {
          "match-community-set": {
           "community-set": "COMMUNITY-SET1",
           "match-set-options": "ALL"
          }
         }
        },
        "actions": {
         "accept-route": [
          null
         ]
        }
       },
       {
        "name": "as-path-set1",
        "conditions": {
         "bgp-policy:bgp-conditions": {
          "match-as-path-set": {
           "as-path-set": "AS-PATH-SET1",
           "match-set-options": "ANY"
          }
         }
        },
        "actions": {
         "bgp-policy:bgp-actions": {
          "set-local-pref": 50
         },
         "accept-route": [
          null
         ]
        }
       },
       {
        "name": "reject route",
        "actions": {
         "reject-route": [
          null
         ]
        }
       }
      ]
     }
    }
   ]
  }
 }
}'''

bgp_json_parsed = json.loads(bgp_json)
policy_parsed = json.loads(policy_code)
fictitious_config = [bgp_json_parsed, policy_parsed]
config_per_router = {}
for router in traffic_matrix.DST_ROUTER.drop_duplicates():
    config_per_router[router] = fictitious_config

# Function definition
We define functions that generate "compound widgets". 

I call compound widgets a set or hierarchy of widgets "stored" behind a single layout widget (e.g. box, tabs), which is the one returned in the function.

In [19]:
def ts_widget(df_traffic, aggregation_columns=None, 
                        time_column="TIME", value_column="BW", 
                        top_flows_to_show=5, align_vertically=True):
    '''
    Returns a Box widget containing various widgets used to depict and explore a time series data frame.
    the widget contains:
    - A HTML and a figure widget used to depict the data of the aggregated data frame.
    - A set of check boxes to allow users to select the characteristics shown that are depicted in the figure and table.
    - An update button used to refresh the table and graph widgets when the user changes the checkboxes.
    - A Text box which contains information on the state of the widget (E.g. Processing, Updated, etc.)
    
    The graph and the table are placed horizontally if the align_vertically is False. If selected, a horizontal 
    widget is more compact, but cannot show that much information.
    
    The update function of the update button is a nested function and uses non-local variables.
    '''
    
    # Get the list of aggregation columns. It defaults to any non-time, non-value column in the df.
    if aggregation_columns is None:
        aggregation_columns = list(set(df_traffic.columns) - {time_column, value_column})
        aggregation_columns = sorted(aggregation_columns)
    else:
        aggregation_columns = list(aggregation_columns)
    
    # Defines the layout objects that we will use depending on the widget layout (horizontal or vertical).
    # The values here defined were obtained with trial and error :)
    
    # The only fancy thing here is the use of a variable traffic_traph_box_widget to define whether the 
    # graph and table box is vertical or horizontal
    if align_vertically:
        all_widget_height = "1050px"
        all_widget_width = "1000px"
        
        table_height = "400px"
        table_width = all_widget_width
        
        graph_table_height = '950px'
        
        traffic_traph_box_widget =  widgets.VBox
        figure_size = [ 9.1,  4.8 ]
    else:
        all_widget_height = "700px"
        all_widget_width = "1000px"
        
        graph_table_height = '650px'
        
        table_height = "600px"
        table_width = "500px"
                
        traffic_traph_box_widget =  widgets.HBox
        figure_size = [ 4.8,  4.8 ]
        
    # define the main widget.
    ts_main_widget = widgets.VBox(layout=widgets.Layout(height=all_widget_height, width=all_widget_width))
    
    # the main widget is formed by a control box and the graph_table_box.
    # The control box which contains the check boxes, update butoon and information box.
    # the graph_and table box is self-described.
    control_information_box = widgets.VBox()
    graph_table_box = traffic_traph_box_widget(layout=widgets.Layout(height=graph_table_height, 
                                                                     width=all_widget_width))
    ts_main_widget.children = (control_information_box, graph_table_box)
       
    # Control box
    # the control box is itself formed by:
    # * another box holding the checkboxes
    # * the update button
    # * The information text
    # The first two elements are horitzontally alligned using a box called cbx_update_box
    
    cbx_update_box = widgets.HBox()
    information_widget = widgets.Text(disabled=True, description="State:")
    control_information_box.children = (cbx_update_box, information_widget)
    
    # the cbx is itself divided into the check_box box and the update button.
    # I place the check boxes into their own box to let them have a box space.
   
    check_boxes = {}
    check_boxes_box = widgets.HBox(layout=widgets.Layout(overflow_x='scroll', height='50px', width='850px'))
    for level in aggregation_columns:
        # Create check boxes for each TS characteristic column.
        this_checkbox = widgets.Checkbox(description=level)
        check_boxes_box.children = check_boxes_box.children + (this_checkbox,)
        this_checkbox.value = False
        check_boxes[level] = this_checkbox

    # Update button
    refresh_button = widgets.Button(description="Update")
    cbx_update_box.children = (check_boxes_box, refresh_button)

    # Now, let us finish with the graph and table box.
    fig_prefix_distribution, ax_prefix_distribution = plt.subplots()
    this_canvas = fig_prefix_distribution.canvas
    fig_prefix_distribution.set_size_inches(figure_size)
    this_canvas.figure.set_label("{}".format("Figure"))

    table_box_layout = widgets.Layout(overflow_x='scroll',
                                overflow_y='scroll',
                    #border='3px solid black',
                    width=table_width,
                    height=table_height,
                    flex_direction='row',
                    display='flex')

    table_widget = widgets.HTML()
    table_box_widget = widgets.VBox(children=(table_widget,), layout=table_box_layout)

    graph_table_box.children = (this_canvas, table_box_widget)

    # Finally, define the update function and assign it to the butotn
    def update_compound_widget(caller=None):
        '''
        The update function checks the aggrupation characteristics, calculates the resulting df,
        and updates the table and graph.
        '''

        information_widget.value = "Updating..."
        # find the aggregation level using the check boxes
        aggregation_level = []
        
        for level in check_boxes:
            check_box = check_boxes[level]

            if check_box.value:
                aggregation_level.append(level)

        table_df, graph_df = process_df_for_widget(df_traffic, aggregation_columns=aggregation_level, 
                                                   value_column=value_column, time_column=time_column, 
                                                   top_flows_to_show=top_flows_to_show)
        ax_prefix_distribution.clear()
        aggregation_column_name = next(iter(set(graph_df.columns) - {time_column, value_column}))

        graph_df.plot.area(ax=ax_prefix_distribution)

        ax_prefix_distribution.legend_.remove()
        ax_prefix_distribution.legend(bbox_to_anchor=(0.1, 0.85, 0.9, .105), loc=3,
                   ncol=2, mode=None, borderaxespad=0.1, fontsize=8) 
        ax_prefix_distribution.set_ylabel(value_column)
        table_widget.value = table_df.style.set_table_attributes('class="table"').set_table_styles([hover()]).render()

        information_widget.value = "Redrawing..."
        plt.draw_all()
        information_widget.value = "Done"
        
    refresh_button.on_click(update_compound_widget)
    return ts_main_widget

In [20]:
def load_balancing_windows(df_traffic, interested_links, column_links, column_prefixes, rt, column_time="TIME", column_value="BW"):
    '''
    Returns a compound widget which includes elements to illustrate a load balancing.
    It contains:
    A button to trigger the LB process
    A Figure with the current state (Left)
    A Figure with the resulting state (right)
    A HTML widget with the summary of the changes to achieve the balancing.
    '''
    
    # prepare figures (for both before and after balancing)
    before_fig, before_axes = plt.subplots()
    before_fig.set_size_inches([ 4.6,  3.0 ])
    before_fig_canvas = before_fig.canvas
    
    after_fig, after_axes = plt.subplots()
    after_fig.set_size_inches([ 4.6,  3.0 ])
    after_fig_canvas = after_fig.canvas
    
    figures_box = widgets.HBox(children=(before_fig_canvas, after_fig_canvas), layout=widgets.Layout(width='1000px', height='600px'))
    
    # changes html
    changes_widget = widgets.HTML(layout=widgets.Layout(width='800px'))
    changes_box = widgets.Box(children=(changes_widget,), layout=widgets.Layout(overflow_y="scroll", height="300px"))
    
    # draw before figure
    try:
        before_axes.legend_.remove()
    except:
        pass
    get_total_per_link(df_traffic, interested_links, column_links, column_time, column_value).plot(ax=before_axes)
    
    before_axes.legend(bbox_to_anchor=(0.1, 0.85, 0.9, .105), loc=3,
                   ncol=2, mode=None, borderaxespad=0.1, fontsize=8) 
    
    plt.draw_all()
    
    # prepare buttons and changes
    update_button = widgets.Button(description="Update LB")
    
    button_box = widgets.HBox(children=(update_button,), layout=widgets.Layout(width='800px'))
    def update_button_cb(caller=None):
        changes = load_balancing(df_traffic, interested_links, column_links, column_prefixes, rt, column_time, column_value)
        resulting_df = apply_changes(df_traffic, changes, column_links, column_prefixes)
        changes_widget.value = summarize_change_html(changes)
        get_total_per_link(resulting_df, interested_links, column_links, column_time, column_value).plot(ax=after_axes)
        after_axes.set_ylim(before_axes.get_ylim())
        plt.draw_all()
    update_button.on_click(update_button_cb)
    
    # build the final resulting widget
    lb_widget = widgets.VBox(children=(button_box, figures_box, changes_box), 
                             layout=widgets.Layout(width='1000px', height='700px', overflow_y='scroll'))
    return lb_widget
  
#plt.close("all")
#interested_links = {'Amsterdam_0', 'Barcelona_0', 'Frankfurt_0'}
#lb_wiget = load_balancing_windows(traffic_matrix, interested_links, column_links="EGRESS_LINK", column_prefixes="DST_NET", rt=routing_table)
#display(lb_wiget)

In [21]:
def json_browser(input_data):
    '''
    The JSON browser receives a structure variable and 
    allows an user to navigate it similarly to the columns view of files in Finder of Mac.
    The idea of the widget is let users navigate a json-type dict object,
    by going deeper in the hierarchies.
    For each selected level, the user can navigate further the elements.
    If the level has any element with a value, it shows them in its own box.
    '''
    
    # Defining overall layout objects.
    height_selection_box = 300
    smal_box_height = height_selection_box * 0.94
    general_framework_layoud = widgets.Layout(overflow_x='scroll',
                            #overflow_y='scroll',
                            #border='3px solid black',
                            #height='',
                            flex_direction='row',
                            display='flex',
                            width='900px',
                            height='{}px'.format(height_selection_box))


    small_box_layout = widgets.Layout(overflow_x=None,
                            #overflow_y='scroll',
                            #border='3px solid black',
                            #height='',
                            #flex_direction='row',
                            #display='flex')
                            width='300px',
                            min_width='300px',
                            min_height='{}px'.format(smal_box_height),
                            height='{}px'.format(smal_box_height))

    divided_box_layout = widgets.Layout(overflow_x=None,
                            #overflow_y='scroll',
                            #border='3px solid black',
                            #height='',
                            #flex_direction='row',
                            #display='flex')
                            width='300px',
                            min_width='300px',
                            height='{}px'.format(int(smal_box_height * 0.49)))

    # Let us define the main compound widget box.
    main_box = widgets.HBox(layout=general_framework_layoud)
    
    # each hierarchy is shown in a selection box. We need to keep track of the information that 
    # each box stores. We define all that information here
    widget_to_data = {}
    keys, processed_data, these_values = process_data(input_data)
    select_to_parent = {}

    # I would normally use Select, but ipywidgets 6.0 uses a list instead of a box
    # this will be fix later, but the 7.0 had other problems when I tested it.
    select_widget = widgets.SelectMultiple(
        #description='',
        options=[None] + list(keys),#ordered_keys,
        rows=10,
        #options=['Linux\ndf', 'Windows', "OSX"],
        #options=range(0, 100),
       layout=small_box_layout,
    )

    widget_to_data[select_widget] = processed_data
    main_box.children = (select_widget,)

    # We then define the update function of the selector, that we will
    # link to an observe callback pointed to the value trait of the select box.

    def handle_change(caller, names="value"):
        select_box_called = caller['owner']
        
        # the "train" box is different from the select box only if there is content
        train_box = select_to_parent.get(select_box_called, select_box_called)
        # change this to value when not s selectmultiple
        #this_key = select_box_called.value
        if select_box_called.value:
            this_key = select_box_called.value[0]
        else:
            this_key = None

        if this_key is not None:
            #index = caller["new"]["index"]
            if select_box_called not in widget_to_data:
                raise Exception("Error. Could not identify the selection box.".format())
            select_box_data = widget_to_data[select_box_called]
            if this_key not in select_box_data:
                raise Exception("Error. Could not identify value {} in data for selection box.".format(this_key))
            new_value = select_box_data[this_key]
        else:
            new_value = None

        keys, processed_data, these_values = process_data(new_value)

        
        if keys is None:
            if processed_data is None:
                # We are in the None line, 'eliminate' the rest of selected boxes
                main_box.children = main_box.children[:main_box.children.index(train_box) + 1] 
            else:
                # we are in a value. Show it.
                new_select_box = widgets.Text(value=processed_data, layout=small_box_layout, disabled=True)
                main_box.children = main_box.children[:main_box.children.index(train_box) + 1] + (new_select_box,)
        else:
            # We need a new selected box. and potentially a text
            if these_values is None or not these_values:
                # We do not need to show any values for this level.
                new_select_box = widgets.SelectMultiple(
                    #description='',
                    options=[None] +  list(keys),#ordered_keys,
                    rows=10,
                    #options=['Linux\ndf', 'Windows', "OSX"],
                    #options=range(0, 100),
                   layout=small_box_layout,
                    )
                new_select_box.observe(handle_change, names="value")

                # the widget_to_data accumulates garbage with time. This is ok for a proto though.
                widget_to_data[new_select_box] = processed_data
                main_box.children = main_box.children[:main_box.children.index(train_box) + 1] + (new_select_box,)
            else:
                # we need to show values for this level.
                new_select_box = widgets.SelectMultiple(
                    #description='',
                    options=[None] +  list(keys),#ordered_keys,
                    rows=10,
                    #options=['Linux\ndf', 'Windows', "OSX"],
                    #options=range(0, 100),
                   layout=divided_box_layout,
                    )
                value_content = '\n'.join(['{}: {}'. format(key, value)for key, value in these_values.items()])

                new_values_box = widgets.Textarea(value=value_content, layout=divided_box_layout, disabled=True)
                new_select_box.observe(handle_change, names="value")
                new_holding_box = widgets.VBox(layout=small_box_layout, children=(new_values_box, new_select_box))

                select_to_parent[new_select_box] = new_holding_box
                # the widget_to_data accumulates garbage with time. This is ok for a proto though.
                widget_to_data[new_select_box] = processed_data
                main_box.children = main_box.children[:main_box.children.index(train_box) + 1] + (new_holding_box,)
                

    select_widget.observe(handle_change, names="value")
    return main_box

# Dashboard elements definition

We define next the elements of the dashboard.

(Dashboard elements are also compound widgets, but I do not generate them, but only create them and display them at the cell)

In [22]:
# This cell displays the tabs used to explore the traffic: Traffic matrix, egress traffic, and ingress traffic.
# Another tab, router configuration, is used to explore some ficticious configuration of the router.

general_layout = widgets.Layout(width='1050px', height='800px', overflow_x='scroll',
                                overflow_y='scroll',)
sub_tabs_layout = widgets.Layout(width='1000px', height='700px', overflow_x='scroll',
                                overflow_y='scroll',)

egress_traffic_box = widgets.Tab(layout=general_layout)
ingress_traffic_box = widgets.Tab(layout=general_layout)
tm_box = widgets.VBox(layout=general_layout)
router_config_box = widgets.Box(layout=general_layout)

tabs_widget = widgets.Tab(children=(tm_box, egress_traffic_box, ingress_traffic_box, router_config_box))
names = ["Traffic Matrix", "Egress", "Ingress", "Router Configuration"]
for tab_n, name in enumerate(names):
    tabs_widget.set_title(tab_n, name)
display(tabs_widget)

In [23]:
# In this cell, we create filtering widget.
# The filtering widget allows the user to select the neighboring ASNs and Routers to be "explored".
# it consists in a couple of multiple selector widgets, and a couple of buttons.
# When the buttons are clicked, the tabs contaning the types of traffic are populated with the resulting df.
# There is also an state textbox used to inform the current filter, and a html widget with some instructions.

df_traffic = traffic_matrix

# The main box is called the box_filters
box_filters = widgets.VBox(layout=widgets.Layout(border='1px solid black', overflow_x='scroll',))

# the main box is compoused by an explanation text, the selector boxes with the filters, 
# the buttons, and the state text.
# Let us start defining the explanation and state widgets (they are simple)
text_explanation = "Select the routers and/or directly connected ASes to analyze."
explanation_text = widgets.HTML(value=text_explanation, layout=widgets.Layout(width='700px'), disabled=True)

information_filter = widgets.Text(value="No Filter", description="Current Filter State", layout=widgets.Layout(width='500px'))
information_filter.disabled = True

# The buttons next
clear_all_button = widgets.Button(description="Clear filter")
filter_update_button = widgets.Button(description="Update filter")

# Now the selector boxes
# I tried to abstract a bit the creation of the boxes just in case I needed to extend them later
# Basically, the for loop under creates two selector boxes based on the info of filters_to_columns
filters_to_columns = {"routers": {"Columns": {"DST_ROUTER", "SRC_ROUTER"}, "Description": "Routers"}, 
                      "asns": {"Columns":{"SRC_AS", "DST_AS"}, "Description": "Connected ASNs"}}


filters = {}
for filter_column in filters_to_columns:
    filter_selector = widgets.SelectMultiple(description=filters_to_columns[filter_column]["Description"], 
                                             options=list(set(df_traffic[list(filters_to_columns[filter_column]["Columns"])].drop_duplicates().values.flatten())))
    filters[filter_column] = filter_selector

# a helper tuple to abstract SOME operations, we still need to know which filter is for routers and ases though.
filter_selectors = tuple(filters.values())

# Let us know assign the children to the main filter box:
# Note that that we use a HBox to allign the selectors with another unnamed Vbox holding the buttons
box_filters.children = (explanation_text, 
                       widgets.HBox(children=filter_selectors + (widgets.VBox(children=(filter_update_button, clear_all_button)),)), 
                        information_filter,)

# Finally, let us define the update and clear functions

def update_filters(caller=None):
    # let us close all figures to preserve memory
    plt.close("all")
    
    # We now go over the filters, processing the main df, until we obtain a resulting df (filtered_df)
    filtered_df = df_traffic
    information_filter.value = "Processing"
    if not any([x.value for x in filter_selectors]):
        information_filter_value = "No Filter"
    else:
        information_texts = []
        for filter_column in filters:
            filtered_values = set(filters[filter_column].value)
            if filtered_values:
                filtered_column = np.zeros(len(filtered_df), dtype=bool)
                for column in filters_to_columns[filter_column]["Columns"]: 
                    filtered_column = filtered_column | filtered_df[column].isin(filtered_values).values
                filtered_df = filtered_df[filtered_column]
                information_texts.append("{} {}".format(filter_column, filtered_values))

        information_filter_value = "Traffic from: " + "; ".join(information_texts)
    
    # Based on the filtered_df we create all widgets
    tm = get_traffic_matrix(filtered_df)
    tm_widget = ts_widget(tm, align_vertically=False)
    tm_box.children = (tm_widget,)
    
    selected_routers = set(filtered_df.DST_ROUTER.drop_duplicates().values) | set(filtered_df.SRC_ROUTER.drop_duplicates().values)
    router_config_widget = json_browser({ router:config_per_router[router] for router in selected_routers if router in config_per_router})
    router_config_box.children = (router_config_widget,)
                                         
    asn_filters = set(filters["asns"].value)
    
    egress_tr = get_egress_traffic(filtered_df, asns=asn_filters)
    egress_expl_widget = ts_widget(egress_tr, align_vertically=False)
    interested_links = egress_tr.EGRESS_LINK.drop_duplicates()
    egress_lb_widget = load_balancing_windows(traffic_matrix, interested_links, 
                                              column_links="EGRESS_LINK", column_prefixes="DST_NET", rt=routing_table)
    egress_traffic_box.children = (egress_expl_widget, egress_lb_widget)
    
    names = ["Explore", "Load Balancing"]
    for tab_n, name in enumerate(names):
        egress_traffic_box.set_title(tab_n, name)
    
    ingress_tr = get_ingress_traffic(filtered_df, asns=asn_filters)
    ingress_expl_widget = ts_widget(ingress_tr, align_vertically=False)
    interested_links = ingress_tr.INGRESS_LINK.drop_duplicates()
    ingress_lb_widget = load_balancing_windows(traffic_matrix, interested_links, 
                                              column_links="INGRESS_LINK", column_prefixes="SRC_NET", rt=routing_table)
    ingress_traffic_box.children = (ingress_expl_widget, ingress_lb_widget)
    
    names = ["Explore", "Load Balancing"]
    for tab_n, name in enumerate(names):
        ingress_traffic_box.set_title(tab_n, name)
        
    information_filter.value = information_filter_value
    
def clear_filters(caller=None):
    # the clear button clears all filters and updates the exploration widgets.
    for filter_selector in filter_selectors:
        filter_selector.value = []
    #update_filters(caller=None)

filter_update_button.on_click(update_filters)   
clear_all_button.on_click(clear_filters)


display(box_filters)

In [24]:
extended_link = traffic_matrix.copy()
extended_link["EXTENDED_INGRESS_LINK"] = extended_link.INGRESS_LINK + "_" + extended_link.SRC_AS.astype(str)
ingress_link_bw = extended_link.groupby("EXTENDED_INGRESS_LINK")["BW"].sum()
extended_link["EXTENDED_EGRESS_LINK"] = extended_link.EGRESS_LINK + "_" + extended_link.DST_AS.astype(str)
egress_link_bw = extended_link.groupby("EXTENDED_EGRESS_LINK")["BW"].sum()

In [25]:
ingress_summary_box = widgets.Box()
egress_summary_box = widgets.Box()
tab_totals_traffic = widgets.Tab(children=(egress_summary_box, ingress_summary_box), _titles={0: "Egress", 1: "Ingress"})
#ingress_fig, ingres_axis = plt.subplots()
#ingress_link_bw.sort_values()[0:3].plot.barh(ax=ingres_axis)
#ingress_fig.set_size_inches([3,  1])
#ingress_fig.canvas.layout.width = "800px"
ingress_bw_html = widgets.HTML()
ingress_bw_html.value = ingress_link_bw.reset_index().sort_values("BW", ascending=False)[0:3].style.set_table_attributes('class="table"').set_table_styles([hover()]).render()
ingress_summary_box.children = (ingress_bw_html, )
tab_totals_traffic = widgets.Tab(children=(egress_summary_box, ingress_summary_box), _titles={0: "Egress", 1: "Ingress"})
#egress_fig, egress_axis = plt.subplots()
#egress_link_bw.sort_values()[0:3].plot.barh(ax=egress_axis)
#egress_fig.set_size_inches([3,  1])
#egress_fig.canvas.layout.width = "800px"
egress_bw_html = widgets.HTML()
egress_bw_html.value = egress_link_bw.reset_index().sort_values("BW", ascending=False)[0:3].style.set_table_attributes('class="table"').set_table_styles([hover()]).render()
egress_summary_box.children = (egress_bw_html, )
egress_summary_box.children = (egress_bw_html, )
display(tab_totals_traffic)